In [ ]:
#Secventa de incarcare si dezarhivare a imaginilor
# from google.colab import drive
# drive.mount('/content/gdrive')
# !unzip  gdrive/My\ Drive/unibuc-brain-ad.zip

import tensorflow as tf
from tensorflow.keras import layers
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from tensorflow.keras.metrics import AUC
from sklearn.metrics import confusion_matrix
from sklearn import metrics

#functia care imi proceseaza imaginile, in functie de ce lungime are numarul imaginii, stiu cati de 0 se afla
#in numele png-ului. folosesc convert("L") pentru a converti imaginile la formatul alb-negru
def process_images(start, end):
  lst = []
  for i in range(start,end):
    if len(str(i)) == 1:
      img = Image.open("data/data/00000" + str(i) + ".png").convert("L")
    elif len(str(i)) == 2:
      img = Image.open("data/data/0000" + str(i) + ".png").convert("L")
    elif len(str(i)) == 3:
      img = Image.open("data/data/000" + str(i) + ".png").convert("L")
    elif len(str(i)) == 4:
      img = Image.open("data/data/00" + str(i) + ".png").convert("L")
    elif len(str(i)) == 5:
      img = Image.open("data/data/0" + str(i) + ".png").convert("L")
    img = np.array(img)
    #micsorarea dimensiunii imaginii
    img = img[18:194]
    img = img[:, 30:190]
    #normalizarea imaginii
    normalized_image = preprocessing.normalize(img, norm = "l2")
    lst.append(normalized_image)
  return lst

#creare imaginile de antrenare, care stiu ca sunt de la 1 la 15000
#imaginilor le fac reshape pentru a le putea da ca parametru modelului de CNN. 1 este canalul de culoare
#deoarece imaginile sunt alb-negru

test_images = np.array(process_images(17001,22150))
test_images = test_images.reshape(5149, 176, 160, 1)

#creare imaginile de validare, in numar de 2000, de la 15001 la 17000train_images = np.array(process_images(1,15001))
train_images = train_images.reshape(15000, 176, 160, 1)

#creare imaginile de testare, 5149 care au mai ramas
validation_images = np.array(process_images(15001,17001))
validation_images = validation_images.reshape(2000, 176, 160, 1)

#procesarea label-urilor
def process_labels(start, end, file):
    #imi initializez astfel label-urile deoarece nu sunt unidimensionale
    #daca o imagine apartine clasei 0, label-ul sau este [1,0], daca apartine clasei 1, label-ul este [0,1]
  labels = np.array([np.array([0,0]) for i in range(start,end)])
  with open(file, 'r') as fin:
    fin.readline()
    for i in range(start,end):
      line = [word for word in fin.readline().strip().split(",")]
      labels[i][int(line[1])] = 1
  return labels

#crearea label-urilor
train_labels = process_labels(0,15000,'/content/data/train_labels.txt')
validation_labels = process_labels(0,2000, '/content/data/validation_labels.txt')

#in model, pe langa straturile tipice convolutionale si de MaxPol2D, am si straturi de BatchNormalization si
#Dropout pentru a evita overfitting-ul
def creare_model():
  model = tf.keras.models.Sequential([
      layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
      layers.BatchNormalization(),
      layers.MaxPooling2D(pool_size=(2, 2)),

      layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
      layers.BatchNormalization(),
      layers.MaxPooling2D(pool_size=(2, 2)),
      layers.Dropout(0.1),

      layers.Conv2D(filters=256, kernel_size=(3,3), activation='relu'),
      layers.BatchNormalization(),
      layers.MaxPooling2D(pool_size=(2, 2)),
      layers.Dropout(0.1),

      layers.Conv2D(filters=512, kernel_size=(3,3), activation='relu'),
      layers.BatchNormalization(),
      layers.MaxPooling2D(pool_size=(2, 2)),
      layers.Dropout(0.2),

      layers.Flatten(),
      layers.Dense(512, activation='relu'),
      layers.BatchNormalization(),
      layers.Dropout(0.2),
      layers.Dense(256, activation='relu'),
      layers.BatchNormalization(),
      layers.Dropout(0.2),
      layers.Dense(128, activation='relu'),
      layers.BatchNormalization(),
      layers.Dropout(0.2),
      layers.Dense(64, activation='relu'),
      layers.BatchNormalization(),
      layers.Dropout(0.2),
      layers.Dense(2, activation='sigmoid')
  ])

  return model

numar = 0
total_corect = 0
scor_anterior = 0
fisiere = []
best_scores = []
count = 0

#functia de salvare a celor 5 modele cele mai performante
class Save_Best(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        global numar
        global test_images
        global total_corect
        global scor_anterior
        global fisiere
        global best_scores
        global count
        #obtin predictiile pentru imaginile de test
        predictions = self.model.predict(test_images)
        #obtin predictiile pentru imaginile de validare
        predictions_validation = self.model.predict(validation_images)
        #obtin acuratetea si scorul AUC
        evaluare = self.model.evaluate(validation_images, validation_labels)
        #calculez matricea de confuzie si o si afisez
        conf_matrix = confusion_matrix([np.argmax(pred) for pred in validation_labels], [np.argmax(pred) for pred in predictions_validation])
        print(conf_matrix)

        #calculez cati de 1 am in predictiile pentru imaginile de test
        counter = 0
        for prediction in predictions:
          if np.argmax(prediction) == 1:
            counter += 1
        print(counter)

        #in aceste conditii, salvez un model
        if conf_matrix[1][1] >= 100 and evaluare[2] >= 0.9:
            #daca inca nu am 5 modele salvate, il salvez oricum
            if len(best_scores) < 5:
              count += 1
              nume_fisier = "model" + str(count) + ".h5"
              self.model.save(nume_fisier)
              #updatez lista cu fisiere si lista cu cele mai bune scoruri
              fisiere.append(nume_fisier)
              best_scores.append(evaluare[2])
              print(f'Salvat model cu {counter} de 1 in output.')
            else:
                #daca deja am salvat 5 modele,
                #in acest caz salvez un model doar daca este mai bun decat cel mai slab model salvat deja
              if evaluare[2] > min(best_scores):
                count += 1
                #gasesc indexul minimului
                index = best_scores.index(min(best_scores))
                #inlocuiesc scorurile si numele fisierului de la acea pozitie
                best_scores[index] = evaluare[2]
                nume_fisier = "model" + str(count) +".h5"
                fisiere[index] = nume_fisier
                self.model.save(nume_fisier)
                print(f'Salvat model cu {counter} de 1 in output.')

model_curent = creare_model()
model_curent.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=[AUC(), 'accuracy'])


#antrenez pentru 50 de epoci
model_curent.fit(train_images, train_labels, epochs=50, callbacks=[Save_Best()])

#functia de incarcare a celor 5 modele salvate
def load_models(fisiere):
  total_predictii = []
  for fisier in fisiere:
      #incarcare model
    model_curent.load_weights(fisier)
    predictii_test = [np.argmax(pred) for pred in model_curent.predict(test_images)]
      #obtin predictiile pe setul de validare pentru a calcula matricea de confuzie pentru acest model
    predictions = model_curent.predict(validation_images)
    display_matrice = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix([np.argmax(pred) for pred in validation_labels], [np.argmax(pred) for pred in predictions]), display_labels = [0,1])
    display_matrice.plot()
    plt.show()

    #afisez recall si precizia pentru modelul curent, pentru ambele clase
    print(metrics.precision_score([np.argmax(pred) for pred in validation_labels], [np.argmax(pred) for pred in predictions], pos_label=0))
    print(metrics.recall_score([np.argmax(pred) for pred in validation_labels], [np.argmax(pred) for pred in predictions],pos_label=0))
    print(metrics.precision_score([np.argmax(pred) for pred in validation_labels], [np.argmax(pred) for pred in predictions], pos_label=1))
    print(metrics.recall_score([np.argmax(pred) for pred in validation_labels], [np.argmax(pred) for pred in predictions], pos_label=1))
    #adaug la predictiile totale predictiile modelului curent
    total_predictii.append(predictii_test)
  return total_predictii

total_predictii = load_models(fisiere)

#calculez predictiile finale prin suma predictiilor de pe fiecare pozitie
suma_pe_coloane = np.sum(total_predictii, axis=0)

#crearea fisierului de output
def create_output_file():
  with open('/content/data/submission.csv', 'w') as fout:
    print("id,class", file = fout)
    image_idx = 17001
    for prediction in suma_pe_coloane:
        #daca macar un model a prezis ca o imagine apartine clasei 1, atunci o incadrez in clasa 1
      if prediction >= 1:
        print("0"+str(image_idx)+",1", file = fout)
      else:
        print("0"+str(image_idx)+",0", file = fout)
      image_idx += 1

create_output_file()